# Multi armed Bandits


In [29]:
import numpy as np
import scipy as sc

class Arms:
    def __init__(self,n,mu):
        self.name = np.linspace(0,n-1,n)
        self.true_val = mu
        
        self.N = np.zeros(n)
        self.val = np.ones(n)
        
        
    def pull_arm(self,arm):
        reward = sc.stats.bernoulli.rvs(self.true_val[arm])
        self.N[arm]+=1
        self.val[arm] = self.val[arm] + ()/N
        return reward

In [30]:
# define arms

n = 5
arms = Arms(n,np.random.rand(n))

In [33]:
arms.pull_arm(3)

1

In [34]:
arms.name

array([0., 1., 2., 3., 4.])

In [35]:
arms.true_val

array([0.1580467 , 0.87484062, 0.8377322 , 0.9073128 , 0.92945598])

In [36]:
arms.N

array([0., 0., 0., 3., 0.])